`local score` vs `loss`

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import sys
import numpy as np
import pandas as ps

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset 


from l5kit.geometry import transform_points
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv

sys.path.append("..")

from src.batteries import t2d, load_checkpoint
from src.batteries.progress import tqdm
from src.models.genet import genet_normal
from src.models.resnets import resnet18, resnet18_cat
from src.models import ModelWithConfidence
from src.criterion import neg_multi_log_likelihood_batch
from src.datasets import CubicAgentDataset

In [3]:
!ls ../data

aerial_map
lyft-motion-prediction-autonomous-vehicles.zip
meta.json
multi_mode_sample_submission.csv
scenes
semantic_map
single_mode_sample_submission.csv


In [4]:
DATA_DIR = "../data"
# DATA_DIR = "../data/scenes/validate_chopped_10"
# os.environ["L5KIT_DATA_FOLDER"] = DATA_DIR

In [5]:
cfg = {
    "format_version": 4,
    "model_params": {
        "history_num_frames": 10,
        "history_step_size": 1,
        "history_delta_time": 0.1,
        "future_num_frames": 50,
        "future_step_size": 1,
        "future_delta_time": 0.1,
    },
    "raster_params": {
        "raster_size": [384, 384],
        "pixel_size": [0.5, 0.5],
        "ego_center": [0.25, 0.5],
        "map_type": "py_semantic",
        "satellite_map_key": "aerial_map/aerial_map.png",
        "semantic_map_key": "semantic_map/semantic_map.pb",
        "dataset_meta_key": "meta.json",
        "filter_agents_threshold": 0.0,
    },
}

In [6]:
history_n_frames = cfg["model_params"]["history_num_frames"]
future_n_frames = cfg["model_params"]["future_num_frames"]
n_trajectories = 3

model = ModelWithConfidence(
    backbone=resnet18(
        pretrained=False,
        in_channels=3 + 2 * (history_n_frames + 1),
        num_classes=2 * future_n_frames * n_trajectories + n_trajectories,
    ),
    future_num_frames=future_n_frames,
    num_trajectories=n_trajectories,
)

# load_checkpoint("../logs/resnet18_bigerimages_continue3_alternative/stage_0/best.pth", model)

load_checkpoint("../logs/resnet18_bigerimages_continue4/stage_0/best.pth", model)

model = model.eval()

<= Loaded model from '../logs/resnet18_bigerimages_continue4/stage_0/best.pth'
Stage: stage_0
Epoch: 1
Metrics:
{'train': {'loss': 12.515466623828052}, 'valid': {'loss': 12.515466623828052}}


In [7]:
valid_mask = np.load(f"{DATA_DIR}/scenes/validate_chopped_100/mask.npz")["arr_0"]

print(valid_mask.shape)

(125423254,)


In [8]:
# dm = LocalDataManager(None)
dm = LocalDataManager(DATA_DIR)
rasterizer = build_rasterizer(cfg, dm)

valid_zarr = ChunkedDataset(dm.require("scenes/validate_chopped_100/validate.zarr")).open()

bs = 32

valid_dataset = CubicAgentDataset(cfg, valid_zarr, rasterizer, agents_mask=valid_mask)
print(len(valid_dataset))

# valid_dataset = Subset(valid_dataset, list(range(bs * 4)))

valid_dataloader = DataLoader(
    valid_dataset,
    shuffle=False,
    batch_size=bs,
    num_workers=20,
)

94694


In [9]:
# print(valid_dataset)

In [10]:
device = torch.device("cuda:0")
model = model.to(device)

In [11]:
# model.eval()
# losses = 0.0
# targets_list = []
# with torch.no_grad(), tqdm(total=len(valid_dataloader), desc="valid") as progress:
#     for idx, batch in enumerate(valid_dataloader):
#         images, targets, target_availabilities = t2d(
#             (
#                 batch["image"],
#                 batch["target_positions"],
#                 batch["target_availabilities"],
#             ),
#             device,
#         )

#         preds, confidences = model(images)
#         loss = neg_multi_log_likelihood_batch(targets, preds, confidences, target_availabilities, reduction="mean")
        
#         losses += loss.item()
        
#         targets_list.append(targets.cpu().detach().numpy())

#         progress.update(1)

# losses /= idx + 1

# print(losses)

In [13]:
model.eval()
torch.set_grad_enabled(False)

# store information for evaluation
future_coords_offsets_pd = []
ground_truth = []
timestamps = []
confidences_list = []
agent_ids = []

with tqdm(total=len(valid_dataloader)) as progress:
    for batch in valid_dataloader:
        inputs = batch['image'].to(device)
        squares = batch["square_category"].to(device)
        months = batch["time_month"].to(device)
        weekdays = batch["time_weekday"].to(device)
        hours= batch["time_hour"].to(device)
        

        preds, confidences = model(
            inputs, #squares, months, weekdays, hours
        )
        
        # TODO: fix coordinates
        _gt = batch["target_positions"].cpu().numpy().copy()
        preds = preds.cpu().numpy().copy()
        world_from_agents = batch["world_from_agent"].numpy()
        centroids = batch["centroid"].numpy()
        for idx in range(len(preds)):
            for mode in range(n_trajectories):
                # FIX
                preds[idx, mode, :, :] = transform_points(preds[idx, mode, :, :], world_from_agents[idx]) - centroids[idx][:2]
            _gt[idx, :, :] = transform_points(_gt[idx, :, :], world_from_agents[idx]) - centroids[idx][:2]
        
        future_coords_offsets_pd.append(preds.copy())
        confidences_list.append(confidences.cpu().numpy().copy())
        timestamps.append(batch["timestamp"].numpy().copy())
        agent_ids.append(batch["track_id"].numpy().copy())

        progress.update(1)


predictions_file = "findings/preds_validate_chopped_100.csv"
write_pred_csv(
    predictions_file,
    timestamps=np.concatenate(timestamps),
    track_ids=np.concatenate(agent_ids),
    coords=np.concatenate(future_coords_offsets_pd),
    confs=np.concatenate(confidences_list)
)

# predictions_file = "findings/target.csv"
# write_pred_csv(
#     predictions_file,
#     timestamps=np.concatenate(timestamps),
#     track_ids=np.concatenate(agent_ids),
#     coords=np.concatenate(_gt),
#     confs=np.ze
# )

100%|████████████████████| 2960/2960 [16:11<00:00,  3.05it/s]


In [14]:
# pred_df = ps.read_csv("findings/preds.csv")[["timestamp", "track_id"]]

# df = ps.read_csv("../data/scenes/validate_chopped_100/gt.csv")
# # df = df.merge(pred_df, how="inner", on=["timestamp", "track_id"])


# # df.to_csv("findings/target.csv", index=False)
# print(df.shape)
# df.head()

In [15]:
from l5kit.evaluation import compute_metrics_csv
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace

In [16]:
metrics = compute_metrics_csv(
    "../data/scenes/validate_chopped_100/gt.csv",
    "findings/preds_validate_chopped_100.csv",
    [neg_multi_log_likelihood, time_displace],
)

for metric_name, metric_mean in metrics.items():
    print(metric_name, metric_mean)

neg_multi_log_likelihood 19.287209305929156
time_displace [0.04403952 0.06675795 0.08868262 0.11151661 0.1311955  0.15069865
 0.17015585 0.18768874 0.20346123 0.21961397 0.23564831 0.24891603
 0.26367328 0.27850528 0.29341769 0.30630399 0.31880464 0.3309135
 0.34227462 0.35395733 0.36545393 0.37582061 0.38518697 0.39510804
 0.40521266 0.41428842 0.42360768 0.43200236 0.44099895 0.44898405
 0.45789234 0.46660706 0.47523587 0.48436561 0.49374141 0.50496516
 0.51558169 0.52608343 0.53507204 0.54692969 0.55864307 0.56990443
 0.58360669 0.59770403 0.61346369 0.6281375  0.64435533 0.66220401
 0.68220182 0.70156792]
